In [38]:
# open goldlabels and predictions from SVM on dev set (best feature selection)
error_inputfile = 'results/pred_gold_dev.csv'
gold_pred_df = pd.read_csv(error_inputfile, sep="\t")
goldlabels = error_table['gold'].tolist()
predictions = error_table['pred'].tolist()

In [39]:
# open .txt file to easily extract tokens
inputfile = 'data/SEM-2012-SharedTask-CD-SCO-dev-simple.v2.txt'
df = pd.read_csv(inputfile, sep="\t", header=None, names=["Book", "Sent nr", "Token nr", "Token", "Label"])
tokenlist = df['Token'].tolist()
realsents = df['Sent nr'].tolist()
tokennrs = df['Token nr'].tolist()

In [40]:
# new sentence numbers
i = 0
sentnrs = []

for nr in tokennrs:
    if nr == 0:
        i += 1
    sentnrs.append(i)

In [41]:
# get all tokens, labels and predictions in dataframe
d = {'sent nr': realsents, 'newsent nr': sentnrs, 'token nr': tokennrs, 'token': tokenlist, 'true': goldlabels, 'pred': predictions}
error_analysis_df = pd.DataFrame(d)
error_analysis_df

,sent nr,newsent nr,token nr,token,true,pred
0,0,1,0,1.,O,O
1,0,1,1,The,O,O
2,0,1,2,Singular,O,O
3,0,1,3,Experience,O,O
4,0,1,4,of,O,O
...,...,...,...,...,...,...
13562,439,787,9,orthodox,O,O
13563,439,787,10,in,O,O
13564,439,787,11,his,O,O
13565,439,787,12,ritual,O,O


In [42]:
error_analysis_df.to_csv('error_analysis_df.csv', sep='\t', header=True, quotechar='|', index=False)

In [43]:
# get all matches and errors together in lists
matches = [[],[],[],[]]
errors = [[],[],[],[]]
i = 0
for token in tokenlist:
    if goldlabels[i] == predictions[i]:
        matches[0].append(token)
        matches[1].append(goldlabels[i])
        matches[2].append(predictions[i])
        matches[3].append(sentnrs[i])
    if goldlabels[i] != predictions[i]:
        errors[0].append(token)
        errors[1].append(goldlabels[i])
        errors[2].append(predictions[i]) 
        errors[3].append(sentnrs[i])
    i +=1

In [44]:
# create dataframe with all matches
d = {'sent nr': matches[3], 'token': matches[0], 'true': matches[1], 'pred': matches[2]}
matches_df = pd.DataFrame(d)
matches_df

,sent nr,token,true,pred
0,1,1.,O,O
1,1,The,O,O
2,1,Singular,O,O
3,1,Experience,O,O
4,1,of,O,O
...,...,...,...,...
13548,787,orthodox,O,O
13549,787,in,O,O
13550,787,his,O,O
13551,787,ritual,O,O


In [45]:
# create dataframe with all errors
d = {'sent nr': errors[3], 'token': errors[0], 'true': errors[1], 'pred': errors[2]}
errors_df = pd.DataFrame(d)
errors_df

,sent nr,token,true,pred
0,18,incredible,O,B-NEG
1,45,none,O,B-NEG
2,231,neither,B-NEG,O
3,238,no,O,B-NEG
4,249,by,B-NEG,O
5,249,means,I-NEG,O
6,455,none,O,B-NEG
7,462,Save,B-NEG,O
8,524,none,O,B-NEG
9,587,nobody,B-NEG,O


In [46]:
# get matches statistics
prefixes = ['un', 'dis', 'im', 'in', 'non', 'ir']
suffixes = ['less', 'lessly', 'lessness']

matching_tokens = matches[0]
matching_true = matches[1]
matching_pred = matches[2]
matching_sents = matches[3]

affixes = [[],[],[]]
lexicals = [[],[],[]]
outsides = [[],[],[]]

i = 0
for token in matching_tokens:
    if token.startswith('un') or token.startswith('dis') or token.startswith('im') or token.startswith('in') or token.startswith('non') or token.startswith('ir') or token.endswith('less') or token.endswith('lessly') or token.endswith('lessness'):
        if matching_true[i] != 'O':
            affixes[0].append(token)
            affixes[1].append(matching_true[i])
            affixes[2].append(matching_sents[i])
        else:
            outsides[0].append(token)
            outsides[1].append(matching_true[i])
            outsides[2].append(matching_sents[i])
    else:
        if matching_true[i] == 'O':
            outsides[0].append(token)
            outsides[1].append(matching_true[i])
            outsides[2].append(matching_sents[i])
        else:
            lexicals[0].append(token)
            lexicals[1].append(matching_true[i])
            lexicals[2].append(matching_sents[i])
    i += 1


In [47]:
# stats
print(f"Matches O total: {len(outsides[0])}")
print(f"Matches ALL NEGS affixal: {len(affixes[0])}")
print(f"Matches ALL NEGS lexical: {len(lexicals[0])}")

Matches O total: 13382
Matches ALL NEGS affixal: 33
Matches ALL NEGS lexical: 138


In [48]:
# more stats (affixal)
b_neg_affix = []
i_neg_affix = []

i = 0
for item in affixes[0]:
    if affixes[1][i] == 'B-NEG':
        b_neg_affix.append((item, affixes[1][i], affixes[2][i]))
    elif affixes[1][i] == 'I-NEG':
        i_neg_affix.append((item, affixes[1][i], affixes[2][i]))
    i +=1

print(f"Matches B-NEG affixal: {len(b_neg_affix)}")
print(f"Matches I-NEG affixal: {len(i_neg_affix)}")


Matches B-NEG affixal: 33
Matches I-NEG affixal: 0


In [49]:
# list of tuples of correctly predicted affixal negation cues and their labels + new sentence numbers
b_neg_affix

[('unpleasant', 'B-NEG', 37),
 ('improper', 'B-NEG', 39),
 ('unbrushed', 'B-NEG', 52),
 ('unshaven', 'B-NEG', 52),
 ('unbrushed', 'B-NEG', 60),
 ('unkempt', 'B-NEG', 60),
 ('unconventional', 'B-NEG', 61),
 ('impatience', 'B-NEG', 122),
 ('unknown', 'B-NEG', 168),
 ('unburned', 'B-NEG', 184),
 ('undoubtedly', 'B-NEG', 207),
 ('uncommonly', 'B-NEG', 225),
 ('impossible', 'B-NEG', 246),
 ('unmistakable', 'B-NEG', 247),
 ('impossible', 'B-NEG', 249),
 ('impossible', 'B-NEG', 279),
 ('unnatural', 'B-NEG', 283),
 ('irreproachable', 'B-NEG', 304),
 ('insensibly', 'B-NEG', 310),
 ('inadmissable', 'B-NEG', 325),
 ('insufferable', 'B-NEG', 329),
 ('needless', 'B-NEG', 457),
 ('dissatisfied', 'B-NEG', 469),
 ('unknown', 'B-NEG', 535),
 ('inexplicable', 'B-NEG', 540),
 ('sapless', 'B-NEG', 559),
 ('undoubtedly', 'B-NEG', 560),
 ('dislike', 'B-NEG', 575),
 ('invisible', 'B-NEG', 614),
 ('unknown', 'B-NEG', 624),
 ('impossible', 'B-NEG', 625),
 ('fearless', 'B-NEG', 675),
 ('unclean', 'B-NEG', 784)]

In [50]:
# even more stats (lexical)
b_neg_lex = []
i_neg_lex = []

i = 0
for item in lexicals[0]:
    if lexicals[1][i] == 'B-NEG':
        b_neg_lex.append((item, lexicals[1][i], lexicals[2][i]))
    elif lexicals[1][i] == 'I-NEG':
        i_neg_lex.append((item, lexicals[1][i], lexicals[2][i]))
    i +=1

print(f"Matches B-NEG lexical: {len(b_neg_lex)}")
print(f"Matches I-NEG lexical: {len(i_neg_lex)}")

Matches B-NEG lexical: 137
Matches I-NEG lexical: 1


In [51]:
# list of tuples of correctly predicted lexical negation cues and their labels + new sentence numbers
b_neg_lex

[('no', 'B-NEG', 4),
 ('No', 'B-NEG', 24),
 ('not', 'B-NEG', 28),
 ('Never', 'B-NEG', 38),
 ('not', 'B-NEG', 44),
 ('not', 'B-NEG', 44),
 ('no', 'B-NEG', 45),
 ('not', 'B-NEG', 47),
 ('no', 'B-NEG', 51),
 ('without', 'B-NEG', 51),
 ('never', 'B-NEG', 54),
 ('not', 'B-NEG', 62),
 ("n't", 'B-NEG', 87),
 ("n't", 'B-NEG', 87),
 ('no', 'B-NEG', 96),
 ('no', 'B-NEG', 97),
 ('never', 'B-NEG', 97),
 ('neither', 'B-NEG', 123),
 ('nor', 'B-NEG', 123),
 ('not', 'B-NEG', 123),
 ('nothing', 'B-NEG', 126),
 ('no', 'B-NEG', 129),
 ('not', 'B-NEG', 132),
 ('no', 'B-NEG', 140),
 ('no', 'B-NEG', 144),
 ('no', 'B-NEG', 146),
 ('No', 'B-NEG', 150),
 ('never', 'B-NEG', 152),
 ('not', 'B-NEG', 165),
 ('nothing', 'B-NEG', 172),
 ('without', 'B-NEG', 191),
 ('nothing', 'B-NEG', 212),
 ('not', 'B-NEG', 215),
 ('nor', 'B-NEG', 215),
 ('no', 'B-NEG', 219),
 ('no', 'B-NEG', 222),
 ('nor', 'B-NEG', 222),
 ('No', 'B-NEG', 224),
 ('no', 'B-NEG', 224),
 ('nothing', 'B-NEG', 226),
 ('nor', 'B-NEG', 231),
 ('no', 'B-NE

In [52]:
# get sentences for all correctly predicted lexical negations and put in list 
l = []
for token, label, sentnr in b_neg_lex:
    l.append(sentnr)


b_neg_lex_sentences = []

for item in l:
    i = 0
    s = []
    for nr in sentnrs:
        if nr == item:
            s.append(tokenlist[i])
        i += 1
    b_neg_lex_sentences.append(s)
    

In [53]:
# now the lists of sentences and lexical negation cues correspond, and we can use it to analyse the context of the errors and matches.
print(b_neg_lex[0])
print(b_neg_lex_sentences[0])

('no', 'B-NEG', 4)
['He', 'made', 'no', 'remark', ',', 'but', 'the', 'matter', 'remained', 'in', 'his', 'thoughts', ',', 'for', 'he', 'stood', 'in', 'front', 'of', 'the', 'fire', 'afterwards', 'with', 'a', 'thoughtful', 'face', ',', 'smoking', 'his', 'pipe', ',', 'and', 'casting', 'an', 'occasional', 'glance', 'at', 'the', 'message', '.']
